In [ ]:
import vispy
import vispy.visuals.transforms as transforms
import vispy.plot as vp
import numpy as np
import phased_array
vispy.use("jupyter_rfb")

Assumes embedded element pattern is an omnidirectional antenna. Uses equations
in Phased Array Antenna Handbook, 3rd Edition.




$$
F(θ, ϕ) = \sum a_i \exp(jk \pmb{r}_i \cdot \pmb{\hat{r}})
$$
        where

$$
\begin{align*}
k         &= 2 \frac{π}{λ} & \text {wave number} \\
\pmb{\hat{r}}   &= \pmb{\hat{x}} u_0 + \pmb{\hat{y}} + v_0 \pmb{\hat{r}}  + \cos θ_0 & \text{direction of oncoming wave} \\
\pmb{r}_i &= \pmb{\hat{x}} x_i + \pmb{\hat{y}} y_i + \pmb{\hat{z}} z_i & \text{ the position of the $i$dh element} \\
u         &= \sin {θ} \cos {ϕ}  & \text{direction cosine $u$} \\
v         &= \sin{θ} \sin{ϕ} & \text{direction cosine $v$} \\
\\
θ         &= \sin^{-1}(\sqrt{u^2 + v^2}) \\
ϕ         &= \tan^{-1}(\frac{v}{u})
\end{align*}
$$



In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
d = 1.0
n = 8
λ = 2.0
a_i = np.ones(n)
arr = phased_array.PhasedArray.ula(d, n)
x = np.linspace(-90, 90, 300)
θ = np.radians(x)
ϕ = np.zeros_like(θ)
af = arr.array_factor(λ, a_i, θ, ϕ)

fig = vp.Fig()
plot = fig[0, 0]
af_db = 20 * np.log10(np.abs(af))
af_db -= np.max(af_db)
plot.plot((x, af_db), marker_size=0.0, color="blue", width=2.0)
plot.camera.set_range(y=[-40, 0])
fig

In [ ]:
import vispy.visuals as visuals

dx = 1.0
dy = 1.0
λ = 2.0
nx = 8
ny = 8

array = phased_array.PhasedArray.planar(dx, dy, nx, ny)

a_i = np.ones((nx, ny))

MIN=-100
max = 1


n = 200

u = np.linspace(-max, max, n)
v = np.linspace(-max, max, n)

uu, vv = np.meshgrid(u, v)

mag = uu**2 + vv**2

θ = np.arcsin(np.sqrt(uu**2 + vv**2))
ϕ = np.arctan(vv / uu)

af2 = array.array_factor(λ, a_i, θ, ϕ)

fig3 = vp.Fig()
plot3 = fig3[0, 0]
af2_db = 20 * np.log10(np.abs(af2))
af2_db -= np.nanmax(af2_db)
af2_db[af2_db<-40] = -40

surf = plot3.surface(af2_db, x=u, y=v)
# no perspective
surf.transform = transforms.STTransform((10, 10, 1))
plot3.camera.fov = 0
fig3


In [ ]:
af2_db.shape
cmap = vispy.color.get_colormap("viridis")
colors_idx = np.zeros(af2_db.shape + (3,))
# scale between 0 to 1
colors_idx = (af2_db + 40) / 40
colors = cmap[colors_idx.ravel()]
surf.set_data(colors=colors)
# shading: None, "flat", or "smooth"
surf.shading = None